In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import os

import scipy.stats as stats
import warnings

from matplotlib import pyplot as plt
from dotenv import load_dotenv

from scripts.cleaning import remove_outliers

load_dotenv()
plt.style.use('Solarize_Light2')
sns.color_palette("colorblind")

warnings.filterwarnings('ignore')

# Setting default DPI, pulling it from dotenv if it exists, setting it on 100 if not

pc_dpi = int(os.getenv('DPI'))

if pc_dpi is None:
    pc_dpi = 100

# Override max dpi

if pc_dpi > 155:
    pc_dpi = 155


In [ ]:
# File

sea_ds = "./data/seattle_data.csv"

df_seattle = pd.read_csv(sea_ds)


In [ ]:
building_amnt_zero = df_seattle.shape[0]

print(f"There is {building_amnt_zero} buildings in the dataset")


# <u>Analyse exploratoire :</u>

## <u>0. Élimination d'aberrations via IQR</u>

## <u>1. Age des bâtiments </u>
- <u>1.a : Représentation des bâtiments en fonction de leurs dates de construction. </u>
- <u>1.b : Visualisation des statistiques principales de la variable : Énergie par mètre carré </u>
- <u>1.c : Paire 1.a 1.b : Date de construction et utilisation en énergie par mètre carré | Émission de GàES par m2. </u>

<hr>

## <u> 2. Étude de la note Energy Star</u>
- <u> 2.a : Répartition de la note Energy Star au sein des données. </u>
- <u> 2.b : Paire 2 : Note Energy Star en fonction de l'énergie par mètre carré. </u>
- <u> 2.c : Paire 3 : Note Energy Star en fonction de l'intensité d'émissions de GES. </u>
- <u> 2.d : Paire 4 : Energy Star en fonction de l'année de construction. </u>

<hr>

## <u> 3 : Représentation Property Type/Utilisation énergie </u>
- <u> 3.a : Visualisation des catégories principales </u>
- <u> 3.b : Étude en détails de la corrélation taille -> E* </u>
- <u> 3.c : Étude en détails de la corrélation taille -> GES </u>
- <u> 3.d : Étude en détails de la corrélation taille -> EUI </u>

### Export : dataset destine a la prédiction EUI et GES pour feature engineering
_note_ : _Données non standardisées_

### Cette analyse permettra de :
 - Visualiser "l'age" du paysage de Seattle
 - Contextualiser la mesure de la consommation d'énergie au mètre carré, et sa possible relation avec l'age des bâtiments
 - Avoir une meilleure compréhension de la variable Energy Star et de la présenter face à l'age des bâtiments et de leur consommation en énergie


## 0. Elimination d'outliers via la methode interquartile :


In [ ]:
cleaning_list = ["Electricity(kWh)", "SourceEUI(kWh/m2)", "SiteEnergyUse(kWh)"]

for col in cleaning_list:
    remove_outliers(col, df_seattle)

print(df_seattle.shape)

print(f"{building_amnt_zero - df_seattle.shape[0]} buildings deleted")


## 1. Age des batiments

### 1.a : Représentation des bâtiments en fonction de leurs dates de construction.

In [ ]:
data_age = list(df_seattle["YearBuilt"].astype(int))
oldest = min(data_age)
youngest = max(data_age)

print(f"Le plus ancien batiment date de {oldest}, le plus récent (donnees de 2016) date de {youngest}")


In [ ]:
decades = np.arange(1900, 2020, 10)

building_per_decade = dict.fromkeys(decades)

for decade in building_per_decade.keys():
    built = len([year for year in data_age if (year > decade and year < decade + 10)])
    building_per_decade[decade] = built

used_cmap = plt.get_cmap("Dark2")

fig, (ax1, ax2) = plt.subplots(
    ncols=2,
    nrows=1,
    figsize=(22, 10),
    gridspec_kw={"width_ratios": [1, 4]},
    dpi=pc_dpi
    )

ax1.boxplot(data_age, showmeans=True, widths=0.25)

ax2.bar(building_per_decade.keys(), height=building_per_decade.values(), width=8, color=used_cmap.colors)

###
# Titles & Labels
ax1.set_ylabel("Annee de construction")
ax2.set_ylabel("Nombre de batiments construits")
ax2.set_xlabel("Annee de construction")
fig.suptitle("Representation des batiments en fonction de leur annee de construction")
fig.tight_layout()
#
###

plt.show()

#### Observations :

- Le paysage urbain de Seattle date, majoritairement (q1 --> q3) de 1939 - 1990 (moyenne en 1965)
- On remarque une logique économique : pic de construction dans les années 20 (Roaring 20s) suivi par un creux dans les années 30 (Grande Dépression), une grande reprise durant le "_Golden Age of Capitalism_" au début de la guerre froide, et une baisse dans les années 70 (les deux crises pétrolières). Remarque non pertinente à la modélisation mais visualisation intéressante.
- De nombreux projets date de du début du millénaire, la décennie 2010+ n'est pas étudiable : les données s'arrêtent en 2015

### 1.b : Visualisation des statistiques principales de la variable : Energie par metre carré

In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 12),
    dpi=pc_dpi,
    )

source_eui_data = list(df_seattle["SourceEUI(kWh/m2)"].values.astype(float))

ax1 = sns.boxplot(data=source_eui_data, width=0.6, ax=ax1, color="navy", orient="h")


###
# Titles/Lables
ax1.set_xlabel("Consommation d'energie (kWh/m2)")
fig.suptitle("Visualisation de la repartition de l'EUI parmi les données (energy use intensity)")
#
###

plt.show()

print(df_seattle["SourceEUI(kWh/m2)"].astype(float).describe())


#### Analyse :

- 1 : Les outliers sont extrêmement nombreux, uniquement dans les utilisations "hautes"
- 2 : La majorité des bâtiments ont une EUI entre 215 et 494 kWh/m2, 369 en moyenne.
- 3 : L'élimination de bâtiments via la méthode interquartile peut être viable, la réduction de dimension via ACP peut également être une piste


### 1.c : Paire 1 : Date de construction et EUI | GES/m2

In [ ]:
df_seattle.columns

In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 12),
    dpi=pc_dpi,
)

ax1 = sns.barplot(
    x=df_seattle["YearBuilt"].astype(int).values,
    y=df_seattle["SourceEUI(kWh/m2)"].values.astype(float),
    ci=5
)

###
# Titles/Lables
ax1.set_xticklabels([])
ax1.set_ylabel("SourceEUI(kWh/m2)")
ax1.set_xlabel("Année de construction (1900 - 2015)")
fig.suptitle("Visualisation de 'energy use intensity' moyenne en fonction de l'année de construction des batiments")

#
###
fig.tight_layout()
plt.show()


In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 12),
    dpi=pc_dpi,
)

ax1 = sns.barplot(
    x=df_seattle["YearBuilt"].astype(int).values,
    y=df_seattle["GHGEmissionsIntensity(kgCO2e/ft2)"].values.astype(float),
    ci=5
)

###
# Titles/Lables
ax1.set_xticklabels([])
ax1.set_ylabel("GHGEmissionsIntensity(kgCO2e/ft2)")
ax1.set_xlabel("Année de construction (1900 - 2015)")
fig.suptitle("Visualisation de l'emission de GES moyen des batiments\
 en fonction de l'année de construction des batiments")
#
###
fig.tight_layout()
plt.show()


#### Observation : 
- L'age des bâtiments ne semble pas avoir d'impact sur leur intensité d'utilisation énergétique. On peut émettre l'hypothèse que les bâtiments les plus anciens ont été modernisés/rénovés pour avoir une consommation énergétique plus responsable.
- Les bâtiments les plus modernes semblent rejeter légèrement moins de GàES que les bâtiments plus anciens (nouveaux matériaux de construction ?)

## 2. Etude de la note Energy Star

### 2.a : Repartition de la note Energy Star au sein des données


In [ ]:
fig, (ax1, ax2) = plt.subplots(
    ncols=2,
    nrows=1,
    figsize=(22, 12),
    gridspec_kw={"width_ratios": [1, 4]},
    dpi=pc_dpi,
)

rated = df_seattle[df_seattle["ENERGYSTARScore"].notna()]
scores = list(rated["ENERGYSTARScore"].astype(int).values)
score_range = range(0, 101, 1)
scores_dict = dict.fromkeys(score_range)

for score in scores_dict.keys():
    scores_dict[score] = scores.count(score)

ax1.boxplot(scores, showmeans=True, widths=0.25)

ax2.bar(
    x=list(scores_dict.keys()),
    height=list(scores_dict.values()),
    color=used_cmap.colors,
    width=1
    )

####
## Titles/Lables
ax1.set_ylabel("Nombre de batiments")
ax1.set_xlabel("Note Energy Star (0-100)")
fig.suptitle("Répartition de la note Energy Star au sein du dataset (2016)")
##
####

fig.tight_layout()

plt.show()

rated["ENERGYSTARScore"].describe()


#### Observations :
- Les notes sont globalement hautes (Q1=53--Q3=89) - Les bâtiments ont le droit d'afficher le label "Energy Star Certified" si leur note est supérieure ou égale à 75 -> Mediane (sources), motivation pour chercher a atteindre ce score.
- Variance haute (26).

### 2.b : Paire 2 : Note Energy Star en fonction de l'energie par metre carré.

In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 10),
    dpi=pc_dpi,
)

ax1 = sns.barplot(
    x=rated["ENERGYSTARScore"].values.astype(int),
    y=rated["SourceEUI(kWh/m2)"].values.astype(float),
    ci=10
)

x_line = np.arange(0, 101, 5)

y_line = []

for x in x_line:
    ratings_x = rated[rated["ENERGYSTARScore"] == x]
    mean_x = ratings_x["SourceEUI(kWh/m2)"].mean(numeric_only=None)
    if pd.isna(mean_x):
        mean_x = rated["ENERGYSTARScore"].quantile(0.25).mean()
    y_line.append(mean_x)

ax1.plot(x_line, y_line)

###
# Titles/Lables
ax1.set_xticks(np.arange(-1, 101, 5))
ax1.set_ylabel("SourceEUI(kWh/m2)")
ax1.set_xlabel("Energy Star Score (0-100)")
fig.suptitle("Visualisation de 'l'energy use intensity' en fonction de la note E* des batiments")
#
###

fig.tight_layout()
plt.show()


#### Observation 
- On peut observer ici un lien clair entre le Score Energy Star (E*) et l'intensité d'utilisation énergétique (EUI). Le lien est logique dans ce cas : [le site internet](https://www.energystar.gov/buildings/) explique que, entre autres, l'utilisation d'énergie est un facteur.
- Le site internet d'E* fourni également une motivation pour l'obtention de cette note : un bâtiment est "Energy Star Certified" lorsque sa note est supérieure ou égale à 75 (ce qui peut expliquer le pic autour de cette note : les développeurs immobiliers peuvent avoir intérêt à obtenir cette certification)

### 2.c : Paire 3 : Note Energy Star en fonction de l'intensité d'émissions de GàES.

In [ ]:
rated.columns

In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 10),
    dpi=pc_dpi,
)

ax1 = sns.barplot(
    x=rated["ENERGYSTARScore"].values.astype(int),
    y=rated["GHGEmissionsIntensity(kgCO2e/ft2)"].values.astype(float),
    ci=10
)

###
# Titles/Lables
ax1.set_xticks(np.arange(-1, 101, 5))
ax1.set_ylabel("GHGEmissionsIntensity(kgCO2e/ft2)")
ax1.set_xlabel("Energy Star Score (0-100)")
fig.suptitle("Visualisation de L'intensité d'emission de GES en fonction de la note E* des batiments")
#
###

fig.tight_layout()
plt.show()


#### Observations :
- Les bâtiments les mieux notés ont une faible intensité d'émission de GàES, même si cela ne semble pas être systématique (note: 31 -> 56)
- Cela ne semble pas être le seul facteur mais peu tout de même distinguer une tendance

### 2.c Paire 4 : Rapport entre E* et année de construction :


In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 12),
    dpi=pc_dpi,
)

years_list = list(df_seattle[df_seattle["YearBuilt"].notna()]["YearBuilt"].values)
year_keys = []
for year in years_list:
    if year not in year_keys:
        year_keys.append(year)

year_keys.sort()

year_dict = dict.fromkeys(year_keys)

for key in year_dict.keys():
    temp = df_seattle[(df_seattle["YearBuilt"] == key) & (df_seattle["ENERGYSTARScore"].notna())]
    scores = [int(i) for i in temp["ENERGYSTARScore"]]
    try:
        mean = sum(scores) / len(scores)
        year_dict[key] = mean
    except ZeroDivisionError:
        year_dict[key] = 0

ax1.bar(
    x=list(year_dict.keys()),
    height=list(year_dict.values()),
    color=used_cmap.colors,
    width=1,
    )

###
# Titles/Lables
ax1.set_yticks(np.arange(0, 100, 5))
ax1.set_ylabel("Note Energie Star Moyenne")
ax1.set_xlabel("Année de construction (1900, 2015)")
fig.suptitle("Représentation de la moyenne E* des batiments en fonction de leur année de construction")
#
###

fig.tight_layout()
plt.show()


#### Observation :
- L'année de construction ne semble pas de lien clair avec la note E* - Cela peut venir, comme écrit plus haut, de travaux de modernisation par exemple.

## 3 : Représentation Property Type/Utilisation Énergie

<i>On cherche à savoir s'il existe des corrélations entre les principaux types de bâtiments et leur note E*| émissions de GàES | Intensité d utilisation énergétique</i>

### 3.a Visualisation des categories principales

In [ ]:
uniques = df_seattle["PrimaryPropertyType"].unique()
unique_ptype_dict = dict.fromkeys(uniques)
for unique in unique_ptype_dict.keys():
    unique_ptype_dict[unique] = len(df_seattle[df_seattle["PrimaryPropertyType"] == unique])

unique_ptype_dict


#### Observations :
- SPS -> Seattle Public School
- K12 -> Shool -> from kindergarten to 12th grade
- Typos, doubles espaces etc.
- Pré groupement via généralisation
- La généralisation permettra de créer des catégories plus inclusives tout en gardant les propriétés principales des bâtiments
- Nécessité de clarification : Bâtiments types cités universitaires & campus comptent comme résidentiels ?

In [ ]:
correction_dict = {
    "Self-Storage Facility\n": "Self-Storage Facility",  # Possibly warehouse
    "Small- and Mid-Sized Office": "Small & Mid-Sized Office",
    "Restaurant\n": "Restaurant",
    "Distribution Center\n": "Supermarket & Grocery store",
    "Distribution Center": "Supermarket & Grocery store",
    "Supermarket/Grocery Store": "Supermarket & Grocery store",
    "Supermarket / Grocery Store": "Supermarket & Grocery store",
    "SPS-District K-12": "K-12 School",
    "Senior Care Community": "Small & Medium medical facility",  # Unsure
    "Laboratory": "Small & Medium medical facility",  # Unsure
    "Medical Office": "Small & Medium medical facility",  # Unsure
    "Small- and Mid-Sized Office": "Small & Mid-Sized Office",
    "Non-Refrigerated Warehouse": "Warehouse",
    "Residence Hall/Dormitory": "Residence Hall",
    "University": "College/University",
    "Retail Store": "Supermarket & Grocery store",
    "Self-Storage Facility": "Warehouse",  # Unsure
}

for key in correction_dict.keys():
    df_seattle["PrimaryPropertyType"].replace(to_replace=key, value=correction_dict[key], inplace=True)

print(df_seattle["PrimaryPropertyType"].value_counts())



In [ ]:
select_six = [
    "Small & Mid-Sized Office", "Large Office", "Warehouse", "Supermarket & Grocery store",
    "K-12 School", "Small & Medium medical facility",
]

# EUI , color code property type, sample based on smallest group size

fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 16),
    dpi=pc_dpi,
)

data_ptype = df_seattle[df_seattle["PrimaryPropertyType"].isin(select_six)]

ax1 = sns.boxplot(
    x=data_ptype["PrimaryPropertyType"],
    y=data_ptype["SourceEUI(kWh/m2)"],
    data=data_ptype
)

###
# Titles/Lables
ax1.set_xticklabels(
    labels=select_six,
    rotation=45
    )
fig.suptitle("Efficacité d'utilisation energetique en fonction de l'utilisation principale des principaux groupes de batiments")

#
###

plt.tight_layout()
plt.show()


In [ ]:
# GES, color code property type, sample based on smallest group size

fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 16),
    dpi=pc_dpi,
)

data_ptype = df_seattle[df_seattle["PrimaryPropertyType"].isin(select_six)]

ax1 = sns.boxplot(
    x=data_ptype["PrimaryPropertyType"],
    y=data_ptype["GHGEmissionsIntensity(kgCO2e/ft2)"],
    data=data_ptype
)

###
# Titles/Lables
ax1.set_xticklabels(
    labels=select_six,
    rotation=45
    )
ax1.set_ylim(0, 5)  # few extremes above
fig.suptitle("Intensité d'emissions de GES en fonction de l'utilisation principale des principaux groupes de batiments")
#
###

plt.tight_layout()
plt.show()


#### Observations :
- On voit que dans les deux cas, la statistique étudiée semble avoir une grande importance sur la répartition de la statistique en question, en particulier : les bâtiments ayant le même type (ici Office) mais pas la même taille (Small+mid_sized / Large)
- Il est possible de réaliser deux ANOVAs pour vérifier ces hypothèses :

### ANOVAs :
- On définit alpha sur 0.05 | 5%
- On formule les hypothèses nulles suivantes : 
  - Les différences observées entre les bâtiments étudiés ne sont pas dus à leurs types respectifs (a : pour ce qui est de l'utilisation énergétique, b : pour ce qui est de leurs émissions de GàES)
- Les criteres de l'ANOVA sont : 
  - 1. La normalité des résidus (test de Shapiro-Wilks sur les résidus)
  - 2. Indépendance des échantillons (ce qui est, ici, le cas)
  - 3. Une variance homogène entre les échantillons (on utilisera le test de Bartlett)
- On cherche a invalider les deux hypothèses nulles (H0a et H0b)

In [ ]:
alpha = 0.05
α = 0.05  # Double definition for unicode possible errors

anova_dict = dict.fromkeys(select_six)

# Dict w/ key value pair -> key = type, value = tuple, tuple[0] = SourceEUI(kWh/m2), tuple[1] = GHGEmissionsIntensity(kgCO2e/ft2)

for key in anova_dict.keys():
    anova_dict[key] = (
        data_ptype[data_ptype["PrimaryPropertyType"] == key]["SourceEUI(kWh/m2)"].values,
        data_ptype[data_ptype["PrimaryPropertyType"] == key]["GHGEmissionsIntensity(kgCO2e/ft2)"].values
    )

print(anova_dict.keys())


### Tests equal var 1 & 2

In [ ]:
bartlett_eui = stats.bartlett(
    anova_dict["Small & Mid-Sized Office"][0],
    anova_dict["Large Office"][0],
    anova_dict["Warehouse"][0],
    anova_dict["Supermarket & Grocery store"][0],
    anova_dict["K-12 School"][0],
    anova_dict["Small & Medium medical facility"][0]
)

print(bartlett_eui)
print(bartlett_eui.pvalue > α)


In [ ]:
bartlett_ghg = stats.bartlett(
    anova_dict["Small & Mid-Sized Office"][1],
    anova_dict["Large Office"][1],
    anova_dict["Warehouse"][1],
    anova_dict["Supermarket & Grocery store"][1],
    anova_dict["K-12 School"][1],
    anova_dict["Small & Medium medical facility"][1]
)

print(bartlett_ghg)
print(bartlett_ghg.pvalue > α)


#### On ne peut pas proceder a une ANOVA dans les deux cas
- Les contraintes de l'ANOVA ne sont pas respectées

Le test de Kruskal-Wallis peut être utilisé par exemple, mais dans ce cas la, on voit clairement que la variable "PrimaryPropertyType" a une influence sur l EUI et intensité d'émission de GàES. Cette variable est par conséquent pertinente pour la création d'un modèle déterminant ces deux facteurs. Cela pourrait faire l'objet d'une future étude approfondie.

In [ ]:
df_seattle


In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 12),
    dpi=pc_dpi,
)

ax1 = sns.barplot(
    x="ENERGYSTARScore",
    y="PropertyArea(SquareMetre)Total",
    data=df_seattle,
    ci=10
)


###
# Titles/Lables
ax1.set_xticks(np.arange(-1, 100, 5))
ax1.set_xlabel("Note Energie Star Moyenne")
ax1.set_ylabel("Taille moyenne des batiments par note E* (m2)")
fig.suptitle("Représentation de la moyenne E* des batiments en fonction de leur taille")
#
#
###

fig.tight_layout()
plt.show()


#### Observations : 
On ne peut pas établir a visuellement une corrélation claire entre la taille d'un bâtiment et sa note. La répartition ne semble obéir à aucune règle de distribution particulière. L'intérêt est la combinaison Type & Size (même type mais bâtiment plus grand = moins efficace énergétiquement)

<hr>

## Export pour feature engineering :
- Pour éviter la redondance d'information du modèle, on exclura les variables permettant de calculer directement l'intensité d'émission de GàES et d utilisation énergétique
- Les valeurs cibles seront : SourceEUI(kWh/m2) & GHGEmissionsIntensity(kgCO2e/ft2)
- Élimination des redondances : PrimaryPropertyType & BuildingType & LargestPropertyUseType (Building type est moins divers et sera exclu)
- PropertyName n'est pas utile

<hr>

#### Le feature engineering nous permettra de préparer ce dataset pour modélisation et prédictions

In [ ]:
df_seattle.columns


In [ ]:
cols_discard = [
        "BuildingType", "PropertyName", "LargestPropertyUseType",  # *1
        "GHGEmissions(MetricTonsCO2e)", "SourceEUIWN(kWh/m2)", "SiteEnergyUse(kWh)", "SiteEnergyUseWN(kWh)", # *2
        ]

columns_keep = [col for col in df_seattle.columns if col not in cols_discard]

df_export_one = df_seattle[columns_keep].copy()
df_raw_data = df_seattle[columns_keep].copy()

# *1 : redundancies or useless : BTypes +|- = PrimaryPropertyType, 
# LargestPropertyUseType overlap w/ PropertyArea(SquareMetre)Total
# name = useless

# *2 Avoiding data leaks

df_raw_data.set_index("OSEBuildingID", inplace=True)

df_raw_data.to_csv("./data/seattle_raw_data.csv", sep=",")
